<a href="https://colab.research.google.com/github/Habibrehman1996/Crewai_almost_all_tools_-_Agents_in_Colab/blob/main/01_Crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import API KEY

In [ ]:
import os
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GOOGLE_API_KEY')
#

#INSTALL CREWAI TOOLS

In [ ]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 27.8 MB/s

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from crewai.flow.flow import Flow, start, listen

class MyFlow(Flow):

  @start()
  def function1(self):
    print("Daniyal")

  @listen(function1)
  def function2(self):
    print("Salman")

obj = MyFlow()
obj.kickoff()

 Flow started with ID: 49dd5987-04c7-4874-94bb-eb465ad3fd45
Daniyal
Salman


#Connect And Import LLM

In [ ]:
from crewai import LLM
llm1 = LLM(
    model="gemini/gemini-2.0-flash"

)
llm1.call("who is the founder of Hamas?")

'Hamas was co-founded in 1987 by Sheikh Ahmed Yassin and Abdel Aziz al-Rantissi, shortly after the start of the First Intifada.\n'

#Embedder

In [ ]:
google_embedder ={
    "provider": "google",
    "config": {
        "model": "models/text-embedding-004",
        "api_key": userdata.get('GOOGLE_API_KEY'),
    }

}

#Crewai Knowledge

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Habib. He is 28 years old and lives in Karachi.He is working as a HOD in Taqwa Schooling System "
string_source = StringKnowledgeSource(
    content=content,
)

# Create an LLM with a temperature of 0 to ensure deterministic outputs
llm = LLM(model="gemini/gemini-2.0-flash", temperature=0)

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder
)

result = crew.kickoff(inputs={"question": "What city does Habib live in and how old is he and what is his disignation?"})

/usr/local/lib/python3.11/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()


# Agent: About User
## Task: Answer the following questions about the user: What city does Habib live in and how old is he and what is his disignation?


# Agent: About User
## Final Answer: 
Habib lives in Karachi, he is 28 years old, and his designation is HOD (Head of Department) in Taqwa Schooling System.




#Crew_Storage_Dir

In [ ]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew2'

In [ ]:
from crewai import LLM

llm1 = LLM(
    model = "gemini/gemini-2.0-flash",
)

#Crew RAG Storage

In [ ]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
#from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage
from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)
# Assemble your crew with memory capabilities
Crew = Crew(
    agents = [agent],
    tasks = [task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"

        )
    ),
    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

#Crew Memory Agent

In [ ]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
# messages = [
#     {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
#     {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
#     {"role": "user", "content": "I am more of a beach person than a mountain person."},
#     {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
#     {"role": "user", "content": "I like Airbnb more."},
# ]
# client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory
# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)
crew.kickoff(inputs={"question": "what is your favorite vacation destination?"})

# Agent: About User
## Task: Answer the following questions about the user: what is your favorite vacation destination?


# Agent: About User
## Final Answer: 
Based on your preference for beach destinations and Airbnb, I would recommend considering a vacation to the Outer Banks in North Carolina. The Outer Banks offers beautiful beaches, a variety of Airbnb rentals, and opportunities for relaxation and water activities.




/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


CrewOutput(raw='Based on your preference for beach destinations and Airbnb, I would recommend considering a vacation to the Outer Banks in North Carolina. The Outer Banks offers beautiful beaches, a variety of Airbnb rentals, and opportunities for relaxation and water activities.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer the following questions about the user: what is your favorite vacation destination?', name=None, expected_output='An answer to the question.', summary='Answer the following questions about the user: what is your...', raw='Based on your preference for beach destinations and Airbnb, I would recommend considering a vacation to the Outer Banks in North Carolina. The Outer Banks offers beautiful beaches, a variety of Airbnb rentals, and opportunities for relaxation and water activities.', pydantic=None, json_dict=None, agent='About User', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=302, prompt_tokens=2

In [ ]:
!pip install -Uq 'crewai[tools]'

#Serper_API_key

In [ ]:
import os
from google.colab import userdata
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

In [ ]:
from pydantic import BaseModel, Field
class TeacherAssignmentOutPut(BaseModel):
  score :int = Field(..., descrition = "student assignment score")
  feedback :str = Field(..., descrition = "Teacher Feedback ")
  question :str = Field(..., descrition = "question")
  answer :str = Field(..., descrition = "python code for markdown format")

#AI Teacher Agent

In [ ]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Process, crew

Teacher = Agent(
    role="AI Teacher",
    goal="You are AI teacher you can check python assingnment code submition.",
    backstory="""You have check student python code submition.""",
    tools=[CodeInterpreterTool()],
    llm=llm
)
Assignment_check = Task(
    description="You are AI teacher you can check python assingnment code submition. question: ""{question}"" student solution: ""{solution}"" ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=Teacher
)

crew = Crew(
    agents=[Teacher],
    tasks=[Assignment_check],
    verbose=True,
    process=Process.sequential,

)
question =" create function add two numbers"

solution = """
def add_two_number(num1 : int, num2 : int):
  return num1 + num2
"""
result = crew.kickoff(inputs={
    "question": question,
     "solution": solution
    })
print(result)

# Agent: AI Teacher
## Task: You are AI teacher you can check python assingnment code submition. question:  create function add two numbers student solution: 
def add_two_number(num1 : int, num2 : int):
  return num1 + num2
 


# Agent: AI Teacher
## Thought: I need to check the student's code for correctness, style, and adherence to the prompt. The prompt asks for a function that adds two numbers. The student provided a function that appears to do just that. I will execute the code with some test values to ensure it works correctly. I will also provide feedback on the code's style and clarity.
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"def add_two_number(num1 : int, num2 : int):\\n  return num1 + num2\\n\\n# Test cases\\nprint(add_two_number(1, 2))\\nprint(add_two_number(5, 5))\\nprint(add_two_number(10, 0))\", \"libraries_used\": []}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Error while fetching server API version: 

In [ ]:
result.to_dict()

{'score': 5,
 'feedback': 'The code is correct and well-written. The function `add_two_number` takes two integers as input and returns their sum, as expected. Good job! You could add a docstring to explain what the function does.',
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int, num2 : int):\n  return num1 + num2'}

In [ ]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
llm1=LLM(model="gemini/gemini-1.5-flash")

#CrewaiTools


In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["OPENAI_API_KEY"] = "Your Key"

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm1
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,
    planning_llm=llm1 # Enable planning feature
)

# Execute tasks
crew.kickoff()

 
[2025-03-05 18:10:28][INFO]: Planning the crew execution
# Agent: Market Research Analyst
## Task: Research the latest trends in the AI industry and provide a summary.Step 1: The Market Research Analyst will utilize the 'Search the internet with Serper' tool.  The search query will be crafted to be highly specific and effective, for example:  "Top 3 AI industry trends 2024", "Most significant advancements in artificial intelligence",  "Breakthroughs in AI research this year".  The analyst will ensure the search query is modified and iterated upon to refine the results, focusing on reputable sources like academic journals, industry publications (e.g., MIT Technology Review, IEEE Spectrum), and news from major technology companies. 
Step 2:  The analyst will carefully review the top 10 results (or adjust the number based on the relevance of initial results) provided by Serper, identifying common themes and unique perspectives across various sources.  They will prioritize high-quality, 

CrewOutput(raw="# AI's Transformative 2024: Three Key Trends Shaping the Future\n\nThe world of artificial intelligence is evolving at an unprecedented pace, bringing with it both incredible opportunities and significant challenges.  2024 is shaping up to be a pivotal year, marked by three major trends that are fundamentally reshaping the AI landscape and influencing its impact on our lives.  From creative tools to ethical considerations, let's explore these defining forces.\n\n\n**The Generative AI Revolution:** Imagine AI that can create anything from poems and code to images and music. This is the power of generative AI, and its rise is nothing short of explosive.  Businesses are leveraging it to boost marketing campaigns, automate content creation, and even discover new medicines. However, this transformative technology also presents serious ethical questions, concerning copyright, misinformation, and potential bias.  Addressing these concerns is critical to ensuring the responsibl

In [ ]:
# prompt: import markdown for jupyter display for upper cell

from IPython.display import Markdown

Markdown("""# Results of CrewAI execution

The code you provided executes a series of tasks using the CrewAI library.  It involves:

1. **Basic CrewAI Setup and Execution:** The code starts with a simple demonstration of CrewAI's `Flow` class, printing "Daniyal" and then "Salman". It then proceeds to utilize a Gemini language model to answer a question.

2. **Agent with Knowledge Source:** An agent is created with a specific goal and backstory. A knowledge source is created using a string containing information about a user named Habib. The agent then answers a question about Habib using this knowledge.

3. **Crew with Memory:** This part of the code creates a Crew with three types of memory: Long Term Memory, Short Term Memory, and Entity Memory.  These memories are stored using different storage mechanisms, potentially using a SQLite database for LTM and Vector databases for STM and EM.

4. **Mem0 Integration:** Mem0 is integrated with the CrewAI pipeline by creating a Mem0 client and then configuring the crew to utilize Mem0 for user memory storage.  This allows the crew to access past interactions with the user to answer questions about their preferences.

5. **AI Teacher Agent with Code Interpreter:** An agent named "AI Teacher" is created to evaluate student Python code.  It utilizes the `CodeInterpreterTool` from the `crewai_tools` library.  A sample question and solution are provided for the AI teacher to assess.

6. **Market Research and Content Creation:** The final section creates two agents: a researcher and a writer. The researcher uses search tools (SerperDevTool and WebsiteSearchTool) to find the latest AI industry trends, and the writer creates a blog post about those trends based on the research. It also leverages existing blog posts in a directory using the DirectoryReadTool and FileReadTool.  The `planning=True` setting enables CrewAI's task planning capabilities.

**To run this code successfully, you need to:**

* **Set up your API keys:** Replace placeholders like `userdata.get('GOOGLE_API_KEY')`, `userdata.get('MEM0_API_KEY')`, `userdata.get('SERPER_API_KEY')`, and `os.environ["OPENAI_API_KEY"]` with your actual API keys.
* **Install necessary libraries:** Ensure that all dependencies are correctly installed in the colab environment.
* **Configure file paths:** Make sure that file paths like `'./my_crew2'` and `'./blog-posts'` are correct and that the `blog-posts` directory exists and contains some blog posts if that is the way you are running it.


**Expected output:**

The code will output the results of each stage of the CrewAI workflow, including the answers to the various questions posed to the agents. Specifically, the AI Teacher's evaluation of the student code, the research summary produced by the researcher, and the final blog post generated by the writer.  The output may vary depending on the LLM and the provided inputs.


This Markdown cell provides a summary of the notebook's code, including key steps, API keys requirements and potential outputs.
""")


# Results of CrewAI execution

The code you provided executes a series of tasks using the CrewAI library.  It involves:

1. **Basic CrewAI Setup and Execution:** The code starts with a simple demonstration of CrewAI's `Flow` class, printing "Daniyal" and then "Salman". It then proceeds to utilize a Gemini language model to answer a question.

2. **Agent with Knowledge Source:** An agent is created with a specific goal and backstory. A knowledge source is created using a string containing information about a user named Habib. The agent then answers a question about Habib using this knowledge.

3. **Crew with Memory:** This part of the code creates a Crew with three types of memory: Long Term Memory, Short Term Memory, and Entity Memory.  These memories are stored using different storage mechanisms, potentially using a SQLite database for LTM and Vector databases for STM and EM.

4. **Mem0 Integration:** Mem0 is integrated with the CrewAI pipeline by creating a Mem0 client and then configuring the crew to utilize Mem0 for user memory storage.  This allows the crew to access past interactions with the user to answer questions about their preferences.

5. **AI Teacher Agent with Code Interpreter:** An agent named "AI Teacher" is created to evaluate student Python code.  It utilizes the `CodeInterpreterTool` from the `crewai_tools` library.  A sample question and solution are provided for the AI teacher to assess.

6. **Market Research and Content Creation:** The final section creates two agents: a researcher and a writer. The researcher uses search tools (SerperDevTool and WebsiteSearchTool) to find the latest AI industry trends, and the writer creates a blog post about those trends based on the research. It also leverages existing blog posts in a directory using the DirectoryReadTool and FileReadTool.  The `planning=True` setting enables CrewAI's task planning capabilities.

**To run this code successfully, you need to:**

* **Set up your API keys:** Replace placeholders like `userdata.get('GOOGLE_API_KEY')`, `userdata.get('MEM0_API_KEY')`, `userdata.get('SERPER_API_KEY')`, and `os.environ["OPENAI_API_KEY"]` with your actual API keys.
* **Install necessary libraries:** Ensure that all dependencies are correctly installed in the colab environment.
* **Configure file paths:** Make sure that file paths like `'./my_crew2'` and `'./blog-posts'` are correct and that the `blog-posts` directory exists and contains some blog posts if that is the way you are running it.


**Expected output:**

The code will output the results of each stage of the CrewAI workflow, including the answers to the various questions posed to the agents. Specifically, the AI Teacher's evaluation of the student code, the research summary produced by the researcher, and the final blog post generated by the writer.  The output may vary depending on the LLM and the provided inputs.


This Markdown cell provides a summary of the notebook's code, including key steps, API keys requirements and potential outputs.


#Create your own Tools


In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
  """Input schema for MyCustumTool."""
  student_name: str = Field(..., description="Name of the student")
  student_roll_no: int = Field(..., description="Student id")

class PiaicStudentCard(BaseTool):
  name: str = "Piaic student card generator"
  description: str = "this function will create Piaic student card"
  args_schema: Type[BaseModel] = MyToolInput

  def _run(self, student_name: str, student_roll_no: int )-> str:
    return f"""PIAIC student card

student name : {student_name}
student roll no : {student_roll_no}
Pakistan zindabad
"""

In [ ]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
  """this function search piaic student fee update, it will required roll no of PIAIC student"""
  #database
  data ={100:'paid',
         200:'unpaid'}

  status = data.get(roll_no)

  if status:
    return {"status": status}
  else:
    return "student not found"

In [ ]:


from crewai import Agent, Task, Crew

# Use the new tool
card = PiaicStudentCard()

piaic_manager = Agent(
    role = "PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory ="""you are a master at understanding people and their preferences.""",
    tools = [card, my_tool],
    verbose = True,
    llm=llm1
)
piaic_card_creater = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question ",
    expected_output="final query answer only",
    agent=piaic_manager  # Fixed: Changed piaic_manager to fee_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creater],
    verbose=True

)
result = crew.kickoff(inputs={
    "query": "i am PIAIC student my name is Habib ur Rehman and my roll number is 96 , generate a card for me."

})
print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'i am PIAIC student my name is Habib ur Rehman and my roll number is 96 , generate a card for me.' you must be know how to answer his question 


# Agent: PIAIC manager
## Using tool: Piaic student card generator
## Tool Input: 
"{\"student_name\": \"Habib ur Rehman\", \"student_roll_no\": 96}"
## Tool Output: 
PIAIC student card

student name : Habib ur Rehman
student roll no : 96
Pakistan zindabad



# Agent: PIAIC manager
## Final Answer: 
PIAIC student card

student name : Habib ur Rehman
student roll no : 96
Pakistan zindabad


PIAIC student card

student name : Habib ur Rehman
student roll no : 96
Pakistan zindabad


In [ ]:
result = crew.kickoff(inputs={
    "query": "i am PIAIC student my name is Habib ur Rehman and my roll number is 200 , can you share my fee updates."

})
print(result)


# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'i am PIAIC student my name is Habib ur Rehman and my roll number is 200 , can you share my fee updates.' you must be know how to answer his question 


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 200}"
## Tool Output: 
{'status': 'unpaid'}


# Agent: PIAIC manager
## Final Answer: 
Your fee status is unpaid.


Your fee status is unpaid.


#Processes

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["OPENAI_API_KEY"] = "Your Key"

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()


researcher = Agent(
    role = "Researcher",
    goal = "conduct through research and analysis on AI and AI agents",
    backstory ="""you are an expert researcher specialize in technology, software enginnering, AI and startups .""",
    allow_delegation = False,
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)
writer = Agent(
    role = "senior_writer",
    goal = "create compelling content abou AI and AI agents",
    backstory ="""you are a senior writer specialize in technology, software enginnering, AI and startups .""",
    allow_delegation = False,

    llm=llm1
)

task = Task(
    description="Generate a list of 5 intresting ideas for an article, then write one captivating paragraph",
    expected_output="5 bullet point each with paragraph and accompaniying notes. ",

)
manager = Agent(
    role = "Project Manager",
    goal = "Efficiently manage the crew ensure hight-quality task completion",
    backstory ="""you are an experience project manager, skilled in ocerseeing complex projects and guiding .""",
    allow_delegation = True,
    llm=llm1
)

crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    verbose=True,
    process=Process.hierarchical,

)
result=crew.kickoff()
from IPython.display import Markdown
Markdown(result.raw)
# print(result)

# Agent: Project Manager
## Task: Generate a list of 5 intresting ideas for an article, then write one captivating paragraph


# Agent: Project Manager
## Thought: ```tool_code
Thought:I need to delegate the task of generating interesting article ideas to the Researcher.  Then I will ask the senior_writer to write a captivating paragraph based on the best idea.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Generate 5 interesting article ideas for a blog post about productivity techniques.\", \"type\": \"str\"}, \"context\": {\"description\": \"The article should focus on practical, actionable productivity tips that resonate with a wide audience.  The ideas should be diverse and cover different aspects of productivity, such as time management, focus, organization, etc.  Please provide a brief explanation of each idea, no more than 2 sentences per idea. The target audience is professionals and students interested in improving their productivity.

/usr/local/lib/python3.11/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.I

* **The Power of Time Blocking:** Divide your day into specific blocks dedicated to particular tasks. This structured approach minimizes context switching and enhances focus, leading to increased productivity.

* **Mastering the Art of Prioritization:** Learn effective prioritization techniques like the Eisenhower Matrix (urgent/important). This helps you focus on high-impact tasks first, maximizing your output and reducing stress.

* **The Pomodoro Technique for Focused Work:** Use the Pomodoro Technique—25 minutes of focused work followed by a 5-minute break—to maintain concentration and prevent burnout. This structured approach can significantly improve your focus and efficiency.

* **Harnessing the Potential of Technology:** Explore productivity apps and tools to streamline your workflow.  Calendar apps, note-taking software, and task managers can help you organize and manage your time effectively.

* **The Importance of Regular Breaks and Self-Care:** Incorporate regular breaks into your schedule to avoid burnout.  Taking short breaks throughout the day to rest and recharge can improve your overall focus and productivity.


**Captivating Paragraph (based on "Mastering the Art of Prioritization"):**

In today's fast-paced world, mastering the art of prioritization isn't just a helpful skill—it's a necessity.  Effective prioritization isn't about cramming more into your day; it's about strategically focusing your energy on what truly matters, leading to dramatically increased efficiency and a significant reduction in stress.  Imagine sailing a ship: without a clear destination and a prioritized course of action, you’re adrift.  Similarly, without prioritizing tasks, you'll be constantly reacting instead of proactively achieving your goals.  One incredibly effective technique is the Eisenhower Matrix, which categorizes tasks based on urgency and importance.  Urgent and important tasks require immediate attention, while important but not urgent tasks should be scheduled.  Urgent but unimportant tasks should be delegated if possible, and tasks that are neither urgent nor important should be eliminated entirely. By strategically employing this simple framework, you can navigate your workload with clarity, focus on high-impact activities, and reclaim valuable time and mental energy, ultimately leading to a more productive and fulfilling life.
```